In [163]:
## strip out extra lines between entries

ft = open("tables.txt", 'r', encoding="utf8")
ft_new = open("tables_clean.txt", 'w', encoding="utf8")

line1 = ft.readline()
line2 = ft.readline()
isempty1 = line1.isspace()
isempty2 = line2.isspace()

ft_new.write(line1)
for line3 in ft:
    isempty3 = line3.isspace()
    if (not isempty2) or isempty1 or isempty3:
        ft_new.write(line2)
    
    line1 = line2
    line2 = line3
    isempty1 = isempty2
    isempty2 = isempty3
    
ft_new.write(line3)

ft.close()
ft_new.close()

In [164]:
## input filename, output dictionary of {flavor: [text as list of lines]} pairs

ft = open("tables_clean.txt", 'r', encoding='utf8')

lines = ft.readlines()

flavor = lines[0].lower()
text = []
flavor_dict = {}

for i in range(1, len(lines)-1):
    cur = lines[i].rstrip()
    
    if cur.isupper() and lines[i-1].isspace() and lines[i+1].isspace():
        flavor_dict[flavor] = text
        flavor = cur.lower()
        text = []
    else:
        text.append(cur)
        
ft.close()

In [165]:
## pull out quotations and recommended dishes and put in separate dictionary
## can we distinguish "dishes" from other quotations?

quotation_dict = {}
for flavor in flavor_dict:
    lines = flavor_dict[flavor]
    q = []
    ftext = []
    quotation_dict[flavor] = {'author': [], 'quotation': []}
    for l in lines:
        if not l:
            if q and "Dishes" not in q: ftext.append(q)
            q = []
        elif "\u2014" in l[0]:
            quotation_dict[flavor]['quotation'].append(q)
            quotation_dict[flavor]['author'].append(l[1:])
            q = []
        else:
            q.append(l)
    flavor_dict[flavor] = ftext

In [166]:
## strip out Flavor Affinities and put in separate dictionary

import itertools

affinity_dict = {}
for flavor in flavor_dict:
    lines = list(itertools.chain.from_iterable(flavor_dict[flavor]))
#     lines = flavor_dict[flavor]
    flaf = []
    ftext = []
    for l in lines:
        if "+" in l:
            flaf.append(l)
        elif "Flavor Affinit" not in l:
            ftext.append(l)
    flavor_dict[flavor] = ftext
    affinity_dict[flavor] = flaf

In [167]:
## strip out "Avoid" and put in separate dictionary

avoid_dict = {}
for flavor in flavor_dict:
    ftext = []
    avoid = []
    is_avoid = False
    for l in flavor_dict[flavor]:
        if is_avoid:
            avoid.append(l)
        elif "AVOID" in l:
            is_avoid = True
        else:
            ftext.append(l)
    flavor_dict[flavor] = ftext
    avoid_dict[flavor] = avoid

In [168]:
## strip out Pairings and put in separate dictionary)

pairing_dict = {}
keywords = ["Season:", "Taste:", "Function:", "Weight:", "Volume:", "Techniques:", "Tips:", "Techniques/Tips:"] 
for flavor in flavor_dict:
    plist = []
    ftext = []
    for l in flavor_dict[flavor]:
        if any(word in keywords for word in l.split()):
            ftext.append(l)
        else:
            plist.append(l)
    pairing_dict[flavor] = plist
    flavor_dict[flavor] = ftext

In [169]:
## serialize dictionaries and save

import pickle

f_affinity = open("affinity.pickle", 'wb')
f_pairing = open("pairing.pickle", 'wb')
f_avoid = open("avoid.pickle", 'wb')
f_quotation = open("quotation.pickle", 'wb')
f_flavor = open("flavor.pickle", 'wb')

pickle.dump(affinity_dict, f_affinity)
pickle.dump(pairing_dict, f_pairing)
pickle.dump(avoid_dict, f_avoid)
pickle.dump(quotation_dict, f_quotation)
pickle.dump(flavor_dict, f_flavor)

f_affinity.close()
f_pairing.close()
f_avoid.close()
f_quotation.close()
f_flavor.close()